<a href="https://colab.research.google.com/github/gazuty/betfair-dashboard/blob/main/Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# --- STEP 1: Master Updater ---

import os, glob, pandas as pd, shutil

def update_betfair_master():
    BASE_FOLDER    = '/content/drive/My Drive/Betfair'
    ARCHIVE_FOLDER = os.path.join(BASE_FOLDER, 'Archive')
    os.makedirs(ARCHIVE_FOLDER, exist_ok=True)

    MASTER_CSV     = os.path.join(BASE_FOLDER, 'Betfair_Master.csv')
    BETTING_PATTERN= os.path.join(BASE_FOLDER, 'BettingPandL*.csv')
    RESULTS_FILE   = os.path.join(BASE_FOLDER, 'Results Summary export 25-05-11 095900.csv')

    # 1️⃣ Load or start fresh
    if os.path.exists(MASTER_CSV):
        df_master = pd.read_csv(MASTER_CSV)
        df_master['Settled date'] = pd.to_datetime(df_master['Settled date'], errors='coerce')
        df_master['Profit_Loss']   = pd.to_numeric(df_master['Profit_Loss'], errors='coerce')
        print(f"✅ Loaded master: {MASTER_CSV} ({len(df_master)} rows)")
    else:
        print("⚠ No master found — starting fresh.")
        df_master = pd.DataFrame()

    # 2️⃣ Find all new files
    new_files = glob.glob(BETTING_PATTERN)
    if os.path.exists(RESULTS_FILE):
        new_files.append(RESULTS_FILE)
    print(f"📂 Found {len(new_files)} new file(s).")
    if not new_files:
        return

    # 3️⃣ Load & detect profit column
    dfs_new = []
    for file in new_files:
        print(f"📥 Processing {os.path.basename(file)}")
        df = pd.read_csv(file)
        # pick any column containing “profit”
        profs = [c for c in df.columns if 'profit' in c.lower()]
        if not profs:
            print(f"⚠ Skipping — no profit column in {os.path.basename(file)}")
            continue
        # prefer AUD if present
        pick = next((c for c in profs if 'aud' in c.lower()), profs[0])
        df['Profit_Loss'] = pd.to_numeric(df[pick], errors='coerce')
        dfs_new.append(df)
        print(f"  ↳ used '{pick}' as Profit_Loss")

    if not dfs_new:
        print("⚠ No valid data to merge.")
        return

    # 4️⃣ Concat + clean dates
    df_new = pd.concat(dfs_new, ignore_index=True)
    df_new['Settled date'] = pd.to_datetime(df_new['Settled date'], errors='coerce')
    df_new.dropna(subset=['Settled date'], inplace=True)

    # 5️⃣ Dedupe via composite key
    df_master['key'] = (
        df_master['Market'].astype(str) + "|" +
        df_master['Settled date'].dt.strftime('%Y-%m-%d %H:%M:%S') + "|" +
        df_master['Profit_Loss'].astype(str)
    )
    df_new['key'] = (
        df_new['Market'].astype(str) + "|" +
        df_new['Settled date'].dt.strftime('%Y-%m-%d %H:%M:%S') + "|" +
        df_new['Profit_Loss'].astype(str)
    )
    df_unique = df_new[~df_new['key'].isin(df_master.get('key', []))]
    print(f"✅ {len(df_unique)} new row(s) to add.")

    # 6️⃣ Merge & save
    if not df_unique.empty:
        df_combined = pd.concat([df_master.drop(columns=['key'], errors='ignore'),
                                  df_unique.drop(columns=['key'])],
                                  ignore_index=True)
        df_combined.to_csv(MASTER_CSV, index=False)
        print(f"✅ Master updated → {len(df_combined)} rows")
    else:
        print("⚠ No additions required.")

    # 7️⃣ Archive processed
    for f in new_files:
        shutil.move(f, os.path.join(ARCHIVE_FOLDER, os.path.basename(f)))
        print(f"📦 Archived {os.path.basename(f)}")

# Run it
update_betfair_master()


✅ Loaded master: /content/drive/My Drive/Betfair/Betfair_Master.csv (18886 rows)
📂 Found 0 new file(s).


In [28]:
# --- STEP 2: Load Master ---

import os, pandas as pd

BASE_FOLDER = '/content/drive/My Drive/Betfair'
MASTER_CSV  = os.path.join(BASE_FOLDER, 'Betfair_Master.csv')

print(f"Loading master from {MASTER_CSV}")
df = pd.read_csv(MASTER_CSV)
df['Settled date'] = pd.to_datetime(df['Settled date'], errors='coerce')
df['Profit_Loss']   = pd.to_numeric(df['Profit_Loss'], errors='coerce')
df.dropna(subset=['Settled date'], inplace=True)

print(f"✅ {len(df)} rows loaded; Profit_Loss: {df['Profit_Loss'].dtype}")


Loading master from /content/drive/My Drive/Betfair/Betfair_Master.csv
✅ 18886 rows loaded; Profit_Loss: float64


In [29]:
# --- STEP 3: Extract Sport, clean Track_Name, and Country ---

import re

df['Sport'] = df['Market'].str.extract(r'^([^/]+)/')[0].str.strip()
racing_df = df[df['Sport'].isin(['Horse Racing', 'Greyhound Racing'])].copy()
racing_df[['Track_Info', 'Event_Description']] = racing_df['Market'].str.extract(r'/\s*(.*?)\s*:\s*(.*)')

def extract_track_and_country(track_info):
    if pd.isna(track_info):
        return pd.Series([None, 'Unknown'])
    if '(' in track_info and ')' in track_info:
        inside = track_info.split('(')[1].replace(')', '').strip()
        country = inside.split()[0]
        track = track_info.split('(')[0].strip()
    else:
        track = track_info.strip()
        country = 'Unknown'
    return pd.Series([track, country])

def clean_track_name(track):
    if pd.isna(track):
        return None
    return re.sub(r'\b\d{1,2}(st|nd|rd|th)?\s\w+\b', '', track).strip()

racing_df[['Track_Name_Raw', 'Country']] = racing_df['Track_Info'].apply(extract_track_and_country)
racing_df['Track_Name'] = racing_df['Track_Name_Raw'].apply(clean_track_name)
df = df.merge(racing_df[['Market', 'Track_Name', 'Country']], on='Market', how='left')
print(f"✅ After feature extraction: {len(df)} rows, Profit_Loss dtype: {df['Profit_Loss'].dtype}")


✅ After feature extraction: 20838 rows, Profit_Loss dtype: float64


In [30]:
# --- STEP 4: Build complete summary tables (daily, cumulative, weekly, monthly, sport, country) ---

df['Day'] = df['Settled date'].dt.date
df['Month'] = df['Settled date'].dt.to_period('M').astype(str)
df['Week Starting'] = (
    df['Settled date'].dt.floor('D') -
    pd.to_timedelta(df['Settled date'].dt.weekday, unit='d')
)

by_day = df.groupby('Day')['Profit_Loss'].sum().reset_index()
by_day = by_day.sort_values('Day').reset_index(drop=True)
by_day['Cumulative_Profit_Loss'] = by_day['Profit_Loss'].cumsum()
by_day['Profit_Loss'] = pd.to_numeric(by_day['Profit_Loss'], errors='coerce').round(2)
by_day['Cumulative_Profit_Loss'] = pd.to_numeric(by_day['Cumulative_Profit_Loss'], errors='coerce').round(2)

by_week = df.groupby('Week Starting')['Profit_Loss'].sum().reset_index()
by_week = by_week.sort_values('Week Starting').reset_index(drop=True)
by_week['Profit_Loss'] = pd.to_numeric(by_week['Profit_Loss'], errors='coerce').round(2)

by_month = df.groupby('Month')['Profit_Loss'].sum().reset_index()
by_month['Profit_Loss'] = pd.to_numeric(by_month['Profit_Loss'], errors='coerce').round(2)

by_sport = df.groupby('Sport')['Profit_Loss'].sum().reset_index()
by_sport['Profit_Loss'] = pd.to_numeric(by_sport['Profit_Loss'], errors='coerce').round(2)

by_country = df.groupby('Country')['Profit_Loss'].sum().reset_index()
by_country['Profit_Loss'] = pd.to_numeric(by_country['Profit_Loss'], errors='coerce').round(2)

sport_daily = {}
for sport in df['Sport'].dropna().unique():
    temp = df[df['Sport'] == sport].groupby('Day')['Profit_Loss'].sum().reset_index()
    temp = temp.sort_values('Day').reset_index(drop=True)
    temp['Cumulative_Profit_Loss'] = temp['Profit_Loss'].cumsum()
    temp['Profit_Loss'] = pd.to_numeric(temp['Profit_Loss'], errors='coerce').round(2)
    temp['Cumulative_Profit_Loss'] = pd.to_numeric(temp['Cumulative_Profit_Loss'], errors='coerce').round(2)
    sport_daily[f"{sport} Daily"] = temp

# Terminal output for validation
print(f"✅ By Day rows: {len(by_day)}, dtype: {by_day['Profit_Loss'].dtype}")
print(f"✅ By Week rows: {len(by_week)}, dtype: {by_week['Profit_Loss'].dtype}")
print(f"✅ By Month rows: {len(by_month)}, dtype: {by_month['Profit_Loss'].dtype}")
print(f"✅ By Sport rows: {len(by_sport)}, dtype: {by_sport['Profit_Loss'].dtype}")
print(f"✅ By Country rows: {len(by_country)}, dtype: {by_country['Profit_Loss'].dtype}")


✅ By Day rows: 193, dtype: float64
✅ By Week rows: 28, dtype: float64
✅ By Month rows: 7, dtype: float64
✅ By Sport rows: 14, dtype: float64
✅ By Country rows: 8, dtype: float64


In [31]:
# --- STEP 6: Track summaries ---
track_df = df[df['Sport'].isin(['Horse Racing', 'Greyhound Racing'])] \
    .groupby(['Sport', 'Track_Name'])['Profit_Loss'].sum().reset_index()

track_df['Profit_Loss'] = track_df['Profit_Loss'].round(2)

tracks = {
    'Top Horse Tracks': track_df.query("Sport == 'Horse Racing'").nlargest(15, 'Profit_Loss'),
    'Bottom Horse Tracks': track_df.query("Sport == 'Horse Racing'").nsmallest(15, 'Profit_Loss'),
    'Top Greyhound Tracks': track_df.query("Sport == 'Greyhound Racing'").nlargest(15, 'Profit_Loss'),
    'Bottom Greyhound Tracks': track_df.query("Sport == 'Greyhound Racing'").nsmallest(15, 'Profit_Loss')
}

track_stats = track_df
print("✅ Track summaries built.")


✅ Track summaries built.


In [32]:
# --- STEP 7: Compute strike rates for Horse Racing and Greyhound Racing with min 50 bets ---

# Filter for racing sports
df_racing = df[df['Sport'].isin(['Horse Racing', 'Greyhound Racing'])].copy()

# Group and compute
strike_df = (
    df_racing.groupby(['Sport', 'Track_Name'])['Profit_Loss']
    .agg(
        total_bets='count',
        wins=lambda x: (x > 0).sum()
    )
    .reset_index()
)

# Calculate strike rate
strike_df['Strike_Rate'] = strike_df['wins'] / strike_df['total_bets']

# Filter for min 50 bets
strike_df_filtered = strike_df[strike_df['total_bets'] >= 50]

# Top/bottom
top_strike = strike_df_filtered.nlargest(10, 'Strike_Rate')
bottom_strike = strike_df_filtered.nsmallest(10, 'Strike_Rate')

# Preview
print("✅ Strike rates computed (min 50 bets).")
print(strike_df_filtered.head())


✅ Strike rates computed (min 50 bets).
              Sport   Track_Name  total_bets  wins  Strike_Rate
0  Greyhound Racing  Albion Park         305   178     0.583607
1  Greyhound Racing   Angle Park         159    90     0.566038
2  Greyhound Racing     Ballarat         218   118     0.541284
3  Greyhound Racing      Bendigo         156    73     0.467949
6  Greyhound Racing   Cannington         351   204     0.581197


In [33]:
# --- STEP 8: Prepare all_sheets for export ---

all_sheets = {
    'By Day':         by_day,
    'By Day Sorted':  by_day,
    'By Week':        by_week,
    'Cumulative':     by_day[['Day','Cumulative_Profit_Loss']].rename(
                         columns={'Cumulative_Profit_Loss':'Cumulative'}),
    'By Month':       by_month,
    'By Sport':       by_sport,
    'By Country':     by_country,
    'Track Stats':    track_stats,
    'Top Strike Rates':    top_strike,
    'Bottom Strike Rates': bottom_strike,
    **tracks,
    **sport_daily
}

print(f"✅ Prepared {len(all_sheets)} tables for Sheets export")


✅ Prepared 28 tables for Sheets export


In [34]:
print("📊 Top Horse Tracks preview:")
print(tracks['Top Horse Tracks'].head())
print("📊 Bottom Horse Tracks preview:")
print(tracks['Bottom Horse Tracks'].head())


📊 Top Horse Tracks preview:
            Sport Track_Name  Profit_Loss
255  Horse Racing  Newcastle      1560.61
315  Horse Racing  Southwell      1551.13
169  Horse Racing  Geraldton      1400.78
57   Horse Racing    Aintree      1270.77
297  Horse Racing   Rosehill      1198.06
📊 Bottom Horse Tracks preview:
            Sport    Track_Name  Profit_Loss
344  Horse Racing  Turfway Park      -336.93
117  Horse Racing      Chepstow      -155.16
215  Horse Racing     Lingfield      -149.08
360  Horse Racing     Wincanton      -146.82
292  Horse Racing         Ripon      -105.37


In [35]:
# --- STEP 9: Export to Google Sheets ---

import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.auth import default
from datetime import date

# 1️⃣ Authenticate
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 2️⃣ Open sheet
SHEET_NAME = 'Betfair Dashboard'
sh = next((s for s in gc.openall() if s.title == SHEET_NAME), None)
if not sh:
    raise Exception(f"Sheet '{SHEET_NAME}' not found.")
print(f"✅ Connected to '{SHEET_NAME}'")

# 3️⃣ Export each DataFrame
for name, df_out in all_sheets.items():
    # Format Profit_Loss
    if 'Profit_Loss' in df_out:
        df_out['Profit_Loss'] = pd.to_numeric(df_out['Profit_Loss'], errors='coerce').round(2)
        df_out['Profit_Loss'] = df_out['Profit_Loss'].apply(
            lambda x: f"{x:.2f}" if pd.notnull(x) else ""
        )
    # Convert week to string
    if 'Week Starting' in df_out:
        df_out['Week Starting'] = df_out['Week Starting'].astype(str)
    # Round other numerics
    for c in df_out.select_dtypes(['float','int']):
        df_out[c] = df_out[c].round(2)

    # Clear or create tab
    try:
        ws = sh.worksheet(name)
        ws.clear()
    except gspread.exceptions.WorksheetNotFound:
        ws = sh.add_worksheet(title=name, rows=1000, cols=20)

    set_with_dataframe(ws, df_out)
    print(f"✅ Uploaded '{name}'")

# 4️⃣ Optional KPI Dashboard
try:
    dash = sh.worksheet('Dashboard')
    dash.clear()
except gspread.exceptions.WorksheetNotFound:
    dash = sh.add_worksheet('Dashboard', rows=10, cols=5)

total_profit = round(df['Profit_Loss'].sum(), 2)
total_bets   = len(df)
best_day     = df.groupby(df['Settled date'].dt.date)['Profit_Loss'].sum().idxmax()
worst_day    = df.groupby(df['Settled date'].dt.date)['Profit_Loss'].sum().idxmin()
kpis = [
    ['Metric','Value'],
    ['Total Profit/Loss', total_profit],
    ['Number of Bets', total_bets],
    ['Best Day', str(best_day)],
    ['Worst Day', str(worst_day)],
    ['Generated on', str(date.today())]
]
dash.update('A1', kpis)
print("✅ Dashboard KPIs updated")


✅ Connected to 'Betfair Dashboard'
✅ Uploaded 'By Day'
✅ Uploaded 'By Day Sorted'
✅ Uploaded 'By Week'
✅ Uploaded 'Cumulative'
✅ Uploaded 'By Month'
✅ Uploaded 'By Sport'
✅ Uploaded 'By Country'
✅ Uploaded 'Track Stats'
✅ Uploaded 'Top Strike Rates'
✅ Uploaded 'Bottom Strike Rates'
✅ Uploaded 'Top Horse Tracks'
✅ Uploaded 'Bottom Horse Tracks'
✅ Uploaded 'Top Greyhound Tracks'
✅ Uploaded 'Bottom Greyhound Tracks'
✅ Uploaded 'Snooker Daily'
✅ Uploaded 'Ice Hockey Daily'
✅ Uploaded 'Horse Racing Daily'
✅ Uploaded 'Golf Daily'
✅ Uploaded 'Politics Daily'
✅ Uploaded 'Tennis Daily'
✅ Uploaded 'Greyhound Racing Daily'
✅ Uploaded 'Football Daily'
✅ Uploaded 'Motor Sport Daily'
✅ Uploaded 'Cricket Daily'
✅ Uploaded 'Darts Daily'
✅ Uploaded 'Basketball Daily'
✅ Uploaded 'American Football Daily'
✅ Uploaded 'Rugby Union Daily'


/tmp/ipython-input-35-2198114328.py:66: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  dash.update('A1', kpis)


✅ Dashboard KPIs updated
